# Node undersampling

1. Truncate nodes to 5 decimals
2. Find duplicates among nodes
3. Keep only the first node in both nodes and roads
4. Scan all roads to remove the duplicated nodes

Note: con 2 cifre decimali la risoluzione è di circa 2km... bene ma non benissimo

In [1]:
import json

In [2]:
nodes = json.load(open('connected_nodes.json', 'r'))
roads = json.load(open('connected_roads.json', 'r'))

In [3]:
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [4]:
#not really general, I make it work here for my points
def truncate(num):
    return int(num * 100) / 100;
#test
truncate(1.2345678987654)

1.23

In [5]:
trunc_node_coords = {node:(0,0) for node in node_coords} #all nodes are associated with coordinates (0,0) initially

for coords in coords_node:
    trunc_coords = (truncate(coords[0]), truncate(coords[1]))
    node = coords_node[coords]
    trunc_node_coords[node] = trunc_coords
    
print(list(trunc_node_coords.items())[55])
print(len(trunc_node_coords))

('4753324725', (12.64, 45.53))
51917


In [6]:
trunc_coords_node = {trunc_node_coords[node]:node for node in trunc_node_coords} #duplicate autoremoval
print(len(trunc_coords_node))
print(list(trunc_coords_node.items())[25])

trunc_list = []
for node in trunc_coords_node.items():
  trunc_list.append({'id':node[1], 'coordinates':node[0]})

print(trunc_list)

2239
((12.25, 45.49), '4680277110')
[{'id': '1376589486', 'coordinates': (12.53, 45.46)}, {'id': '1972158561', 'coordinates': (12.54, 45.47)}, {'id': '3006329507', 'coordinates': (12.52, 45.46)}, {'id': '1945600532', 'coordinates': (12.68, 45.52)}, {'id': '1945967777', 'coordinates': (12.68, 45.51)}, {'id': '4040294741', 'coordinates': (12.67, 45.57)}, {'id': '3206414098', 'coordinates': (12.66, 45.55)}, {'id': '8390550852', 'coordinates': (12.48, 45.61)}, {'id': '2603262992', 'coordinates': (12.47, 45.62)}, {'id': '1997020317', 'coordinates': (12.55, 45.59)}, {'id': '913606265', 'coordinates': (12.49, 45.61)}, {'id': '2158021631', 'coordinates': (12.64, 45.53)}, {'id': '421888985', 'coordinates': (12.64, 45.54)}, {'id': '7333329790', 'coordinates': (12.63, 45.53)}, {'id': '3267957063', 'coordinates': (12.62, 45.53)}, {'id': '8352640236', 'coordinates': (12.31, 45.44)}, {'id': '1331978448', 'coordinates': (12.27, 45.46)}, {'id': '4294887007', 'coordinates': (12.28, 45.5)}, {'id': '1672

In [7]:
#keep track of duplicates for quick substitution
dupl = {} #key = node replaced | value = node to replace it with
for tr_coords in trunc_coords_node:
    for coords in coords_node:
        if (truncate(coords[0]), truncate(coords[1])) == tr_coords:
            dupl.update({coords_node[coords]:trunc_coords_node[tr_coords]})

In [8]:
#write into a json file

with open("nodes_small.json", "w") as f:
    json.dump(trunc_list, f)

In [9]:
print(roads[4])
print(type(roads[4]))

{'p1': '3318826655', 'p2': '1695859083', 'discriminator': '1695859085', 'directed': False, 'sidewalks': [False, False], 'distance': 184.06649866563413}
<class 'dict'>


In [10]:
#change the roads
new_roads = []
for road in roads:
    road['p1'] = dupl[road['p1']]
    road['p2'] = dupl[road['p2']]
    if road['p1'] != road['p2']:
        new_roads.append(road)

In [11]:
len(new_roads)

10138

In [12]:
with open("roads_small.json", "w") as f:
    json.dump(new_roads, f)
    

In [13]:
#check!
for road in new_roads:
    if road['p1'] not in trunc_node_coords or road['p2'] not in trunc_node_coords:
        print("ERROR")

In [14]:
#re index nodes and roads

nodes_small = json.load(open('nodes_small.json', 'r'))
roads_small = json.load(open('roads_small.json', 'r'))

new_nodes = []
new_roads = roads_small

for i in range(len(nodes_small)):
  node = nodes_small[i]
  node_id = node.get("id")
  new_nodes.append({"id":i, "coordinates": node.get("coordinates")})
  for j in range(len(new_roads)):
    road = new_roads[j]
    if (road.get("p1") == node_id):
      road["p1"] = i
    if (road.get("p2") == node_id):
      road["p2"] = i

with open("new_nodes.json", "w") as f:
    json.dump(new_nodes, f)
  

In [15]:
with open("new_roads.json", "w") as f:
    json.dump(new_roads, f)